<p style="float: left;"><a href="upper-type-bounds.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="lower-type-bounds.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>

# Variance annotations

**Variance is how subtyping <span style="color:red">between more complex types</span> relates to subtyping between their components.**

Suppose `A` and `B` are types, and `K<U>` denotes application of a type constructor `K` with type argument `U`. 

The kind of subtyping relationships are:

- **covariant** if `A` $\leq$ `B`, then `K<A>` $\leq$ `K<B>`.
- **contravariant** if `A` $\leq$ `B`, then `K<B>` $\leq$ `K<A>`.
- **variant** if covariant or contravariant;
- **invariant** or nonvariant if not variant.

In [ ]:
class Foo[+A] // A covariant class
class Bar[-A] // A contravariant class
class Baz[A]  // An invariant class

The combination of type parameters and subtyping poses some interesting questions. 
 
- _Should **Stack[String]** be a subtype of **Stack[AnyRef]**?_

Intuitively, this seems OK, since a stack of `String` is a special case of a stack of `AnyRef`.


**In a purely functional world, all types could be covariant. However, the situation changes once we introduce mutable data.**

```java
class Fruit

val x = new Array[String](1)
val y: Array[Any] = x // This would be legit in Java not in Scala
y(0) = new Fruit
```

**In Java, arrays are covariant.** This might seem natural but leads to safety problems that require special _runtime checks_.

**<span style="color:red">We would end up storing a `Fruit` in `x: Array[String]` which clearly violates type soundness.</span>**

## Scala's variance approuch

Scala solves the variance problem statically instead of in runtime.

Covariant type parameters may appear in:

1. The types of values (fields) defined in the class.
   ```scala
   class Box[+A](val value: A) // covariant in A
   ```
3. The result types of methods in the class.
    ```scala
    trait Producer[+A]
      def produce(): A // result type is covariant
    ```
5. Type arguments to other covariant types.
   ```scala
   case class Node[+A](head: A, tail: List[A]) // List is covariant
   ```

However, <span style="color:red">**formal parameter types of methods cannot be covariant**.<span>

For this reason, the following class definition is invalid:

```scala
class Array[+A] {
  def apply(index: Int): A
  def update(index: Int, elem: A) // ❌ not allowed
}
```

Allowing mutation in method `update` would break type safety, **since updating can change the internal state and invalidate the subtyping relationship.**

There are also methods which do not mutate state, but type parameter still appears contravariant position...

```scala
class Stack[+A] {
    def push(x: A): Stack[A] = . . . // Do not mutate the internal state, returns a new stack
}
```

Unlike arrays, **stacks are purely functional data structures and
therefore should enable covariant subtyping**. For solving this problem we can use [lower type parameter bound](lower-type-bounds.ipynb).

<p style="float: left;"><a href="upper-type-bounds.ipynb" target="_blank">Previous</a></p>
<p style="float: right;"><a href="lower-type-bounds.ipynb" target="_blank">Next</a></p>
<p style="text-align:center;">Tour of Scala</p>
<div style="clear: both;"></div>